<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b1804fc32462e4f9278f3de8b22acf591a455dae133472500c5093738dc4cc95
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 09:24:58
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 41.35 K (0.3%)
Current PnL: -22.80 L (-15.19%)
CY Booked + Current PnL: -10.90 L (-7.26%)
-------------------
Total profit:  1.32 L
Total loss:  -24.12 L
-------------------
Total Booked + Current PnL: 16.08 L (13.06%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.58%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.99 L (64.89%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,57.0,X-LC,3.71,213706.0,9206.0,14083.0,0.49,4.50,6.59,11.39,10.0,0.65,1.55,13.86,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,97.65,52.0,M-SC,1.53,85117.0,-15660.0,15755.0,0.67,-15.54,18.51,0.09,245.0,-0.99,0.62,11.83,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.04,208755.0,17013.0,20750.0,0.96,8.87,9.94,19.69,99.0,0.82,1.52,16.27,XY25,NTT,AC
33,ICICIGI,2260.25,-15.47,62.0,X-MC,1.60,145410.0,9277.0,21855.0,5.94,6.81,15.03,22.86,91.0,0.42,1.06,20.29,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,50.0,X-LC,2.98,215389.0,5023.0,23757.0,0.35,2.39,11.03,13.68,37.0,0.21,1.57,19.31,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-15.47,62.0,X-MC,1.60,145410.0,9277.0,21855.0,5.94,6.81,15.03,22.86,91.0,0.42,1.06,20.29,X40,ATH,INSURANCE
47,LTIM,7230.20,1.45,68.0,H-LC,3.38,251348.0,5197.0,74022.0,2.13,2.11,29.45,32.18,16.0,0.07,1.83,38.92,X200,ATH,IT
43,JIOFIN,387.00,-1.72,53.0,X-LC,12.23,267030.0,-335.0,65796.0,1.60,-0.13,24.64,24.48,48.0,-0.01,1.94,54.78,XY24,BTT,FINANCE
25,GREENPANEL,537.00,214.60,48.0,M-SC,3.84,140398.0,-42680.0,121655.0,1.16,-23.31,86.65,43.14,230.0,-0.35,1.02,32.06,XY24,NTT,MISC
85,WIPRO,420.00,-11.52,57.0,M-LC,5.75,155694.0,4749.0,104704.0,1.09,3.15,67.25,72.51,53.0,0.05,1.13,9.36,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMPUS,393.00,-23.39,57.0,M-SC,2.73,159604.0,-8171.0,64017.0,-1.28,-4.87,40.11,33.28,210.0,-0.13,1.16,23.70,XY24,NTT,FOOTWEAR
7,ATULAUTO,844.00,3645.95,51.0,M-SC,4.75,168232.0,-28900.0,118721.0,-1.11,-14.66,70.57,45.57,236.0,-0.24,1.22,19.27,XY24,NTT,AUTO
45,KPIGREEN,731.05,-5.57,43.0,H-SC,6.43,112442.0,-12855.0,71783.0,-1.04,-10.26,63.84,47.03,141.0,-0.18,0.82,40.79,MH,ATH,POWER
3,ALKYLAMINE,4546.37,-15.60,25.0,H-SC,6.75,78496.0,-22467.0,116998.0,-0.84,-22.25,149.05,93.63,148.0,-0.19,0.57,16.92,SR,ATH,CHEMICALS
12,BATAINDIA,2096.00,-38.01,33.0,X-SC,4.43,92636.0,-36034.0,81334.0,-0.72,-28.00,87.80,35.20,219.0,-0.44,0.67,6.33,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.34,148842.0,-378.0,144555.0,0.37,-0.25,97.12,96.62,208.0,-0.00,1.08,61.08,XY25,NTT,FINANCE
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.27,123548.0,212.0,131418.0,-0.16,0.17,106.37,106.72,119.0,0.00,0.90,23.59,AR,ATH,MISC
54,RECLTD,446.00,45.36,46.0,M-LC,6.32,204710.0,1540.0,40594.0,0.51,0.76,19.83,20.74,55.0,0.04,1.49,6.34,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,104.20,57.0,H-SC,11.48,129330.0,-13149.0,32669.0,0.67,-9.23,25.26,13.70,163.0,-0.40,0.94,52.64,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,97.65,52.0,M-SC,1.53,85117.0,-15660.0,15755.0,0.67,-15.54,18.51,0.09,245.0,-0.99,0.62,11.83,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,47.0,H-SC,1.84,220401.0,-49266.0,85604.0,-0.22,-18.27,38.84,13.47,138.0,-0.58,1.60,12.07,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,2.10,139909.0,-35994.0,78013.0,-0.13,-20.46,55.76,23.89,149.0,-0.46,1.02,20.83,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.89,42.0,H-MC,7.03,101593.0,-30242.0,72466.0,0.22,-22.94,71.33,32.03,98.0,-0.42,0.74,13.79,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,46.0,M-LC,6.32,204710.0,1540.0,40594.0,0.51,0.76,19.83,20.74,55.0,0.04,1.49,6.34,XY25,NTT,FINANCE
31,HINDZINC,730.22,35.55,68.0,M-LC,8.86,220850.0,15774.0,96799.0,0.41,7.69,43.83,54.89,52.0,0.16,1.61,33.64,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,57.0,M-LC,5.75,155694.0,4749.0,104704.0,1.09,3.15,67.25,72.51,53.0,0.05,1.13,9.36,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,53.0,M-LC,9.46,156576.0,15736.0,28747.0,0.52,11.17,18.36,31.59,66.0,0.55,1.14,40.62,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,12.67,192035.0,12227.0,100185.0,0.45,6.80,52.17,62.52,88.0,0.12,1.40,43.20,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,68.0,M-LC,8.86,220850.0,15774.0,96799.0,0.41,7.69,43.83,54.89,52.0,0.16,1.61,33.64,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,12.67,192035.0,12227.0,100185.0,0.45,6.80,52.17,62.52,88.0,0.12,1.40,43.20,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,57.0,M-LC,5.75,155694.0,4749.0,104704.0,1.09,3.15,67.25,72.51,53.0,0.05,1.13,9.36,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.27,123548.0,212.0,131418.0,-0.16,0.17,106.37,106.72,119.0,0.00,0.90,23.59,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-49.12,29.0,M-SC,4.05,85950.0,-5548.0,84300.0,-0.69,-6.06,98.08,86.07,223.0,-0.07,0.62,24.81,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,3.31,143730.0,-132793.0,242861.0,0.13,-48.02,168.97,39.81,54.0,-0.55,1.04,0.13,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51421.0,-13585.0,164511.0,0.28,-20.90,319.93,232.18,198.0,-0.08,0.37,0.28,XY24,NTT,MISC
28,HAVELLS,2069.16,-8.45,31.0,X-MC,1.53,226996.0,-20735.0,95792.0,0.41,-8.37,42.20,30.30,92.0,-0.22,1.65,3.81,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-16.17,32.0,X-MC,5.68,176464.0,-24042.0,121195.0,0.35,-11.99,68.68,48.45,90.0,-0.20,1.28,1.76,X40N,ATH,FINANCE
12,BATAINDIA,2096.00,-38.01,33.0,X-SC,4.43,92636.0,-36034.0,81334.0,-0.72,-28.00,87.80,35.20,219.0,-0.44,0.67,6.33,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.47,43.0,H-LC,0.91,155300.0,-30795.0,78271.0,-0.04,-16.55,50.40,25.51,15.0,-0.39,1.13,14.44,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.04,208755.0,17013.0,20750.0,0.96,8.87,9.94,19.69,99.0,0.82,1.52,16.27,XY25,NTT,AC
28,HAVELLS,2069.16,-8.45,31.0,X-MC,1.53,226996.0,-20735.0,95792.0,0.41,-8.37,42.20,30.30,92.0,-0.22,1.65,3.81,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-15.47,62.0,X-MC,1.60,145410.0,9277.0,21855.0,5.94,6.81,15.03,22.86,91.0,0.42,1.06,20.29,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-9.88,52.0,X-LC,2.23,277173.0,11747.0,45041.0,0.77,4.43,16.25,21.40,11.0,0.26,2.01,11.36,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,3.31,143730.0,-132793.0,242861.0,0.13,-48.02,168.97,39.81,54.0,-0.55,1.04,0.13,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51421.0,-13585.0,164511.0,0.28,-20.90,319.93,232.18,198.0,-0.08,0.37,0.28,XY24,NTT,MISC
82,VBL,671.64,-22.00,37.0,X-LC,10.71,281071.0,-34771.0,147422.0,-0.24,-11.01,52.45,35.67,5.0,-0.24,2.04,1.32,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-16.17,32.0,X-MC,5.68,176464.0,-24042.0,121195.0,0.35,-11.99,68.68,48.45,90.0,-0.20,1.28,1.76,X40N,ATH,FINANCE
79,UNITDSPR,1644.00,-14.23,43.0,X-LC,6.36,224735.0,-9409.0,58027.0,0.32,-4.02,25.82,20.77,86.0,-0.16,1.63,2.62,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51421.0,-13585.0,164511.0,0.28,-20.90,319.93,232.18,198.0,-0.08,0.37,0.28,XY24,NTT,MISC
55,RELAXO,1176.00,-41.04,35.0,X-SC,3.73,80417.0,-64743.0,140673.0,0.18,-44.60,174.93,52.31,136.0,-0.46,0.58,7.64,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-38.01,33.0,X-SC,4.43,92636.0,-36034.0,81334.0,-0.72,-28.00,87.80,35.20,219.0,-0.44,0.67,6.33,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.27,46.0,X-SC,1.18,106610.0,4606.0,39318.0,0.73,4.52,36.88,43.06,122.0,0.12,0.77,22.40,X40N,ATH,MISC
32,HONAUT,58357.33,-22.88,46.0,X-SC,2.29,108720.0,-19218.0,66352.0,0.42,-15.02,61.03,36.84,143.0,-0.29,0.79,11.30,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.33,279124.0,-66832.0,133533.0,0.68,-19.32,47.84,19.28,1.0,-0.50,2.03,3.19,X40,ATH,IT
39,INFY,2275.00,-18.88,49.0,X-LC,7.63,316177.0,3289.0,168396.0,-0.37,1.05,53.26,54.87,3.0,0.02,2.30,7.77,X40,BTT,IT
41,ITC,452.00,-39.61,43.0,X-LC,2.83,195404.0,-4734.0,25617.0,0.71,-2.37,13.11,10.44,4.0,-0.18,1.42,3.07,X40,NTT,FMCG
82,VBL,671.64,-22.00,37.0,X-LC,10.71,281071.0,-34771.0,147422.0,-0.24,-11.01,52.45,35.67,5.0,-0.24,2.04,1.32,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,41.0,H-LC,7.44,245592.0,-16027.0,135248.0,-0.06,-6.13,55.07,45.57,7.0,-0.12,1.78,4.50,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.67,80640.0,-32909.0,72963.0,-0.59,-28.98,90.48,35.27,268.0,-0.45,0.59,97.37,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,38.0,H-SC,9.85,89355.0,-8625.0,30229.0,-0.58,-8.80,33.83,22.05,152.0,-0.29,0.65,29.88,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,104.20,57.0,H-SC,11.48,129330.0,-13149.0,32669.0,0.67,-9.23,25.26,13.70,163.0,-0.40,0.94,52.64,OX40N,NTT,BANKS
33,ICICIGI,2260.25,-15.47,62.0,X-MC,1.60,145410.0,9277.0,21855.0,5.94,6.81,15.03,22.86,91.0,0.42,1.06,20.29,X40,ATH,INSURANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,9.95,145350.0,-33721.0,99448.0,-0.04,-18.83,68.42,36.71,135.0,-0.34,1.06,44.22,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.34,148842.0,-378.0,144555.0,0.37,-0.25,97.12,96.62,208.0,-0.00,1.08,61.08,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,9.95,145350.0,-33721.0,99448.0,-0.04,-18.83,68.42,36.71,135.0,-0.34,1.06,44.22,XR,NTT,BANKS
31,HINDZINC,730.22,35.55,68.0,M-LC,8.86,220850.0,15774.0,96799.0,0.41,7.69,43.83,54.89,52.0,0.16,1.61,33.64,X5K,ATH,METALS
50,PGHH,17907.65,-26.71,58.0,X-MC,3.04,212205.0,11385.0,56404.0,-0.69,5.67,26.58,33.76,80.0,0.20,1.54,11.16,X40,ATH,FMCG
63,SHALBY,327.00,1164.75,53.0,M-SC,22.49,172037.0,-9106.0,54570.0,0.40,-5.03,31.72,25.10,235.0,-0.17,1.25,37.87,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.47
1,25,43.97
2,50,75.83


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.36
LC    33.70
MC    21.89
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.63
X40      14.85
XY25     12.14
XR       10.98
X40N     10.07
AR        8.20
OX40N     7.71
X200      1.83
X5K       1.61
SR        1.11
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.60
X-LC    22.56
X-MC    16.15
M-SC    12.70
M-LC     5.37
H-LC     4.74
X-SC     4.61
H-MC     3.81
M-MC     1.62
L-SC     1.45
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.03,-6.23,41.25
IT,13.07,-16.91,80.14
FINANCE,12.21,-12.56,63.59
MISC,6.89,-20.91,87.91
BANKS,6.44,-12.75,72.07
PAINTS,5.60,-19.75,37.60
ELECTRICAL,5.35,-11.78,51.00
INSURANCE,3.89,-3.37,42.01
AUTO,2.83,-47.05,117.35


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3355180.0,22
XR,1399611.0,14
AR,1200061.0,9
X40,794627.0,10
X40N,639528.0,8
OX40N,585668.0,10
XY25,513655.0,8
SR,268141.0,2
X5K,96799.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3476985.0,25
M-SC,1290336.0,15
X-LC,1133522.0,13
X-MC,1104213.0,11
X-SC,544646.0,6
H-MC,355914.0,3
H-LC,287541.0,3
M-LC,270844.0,4
L-SC,262435.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1211538.0      6
           XR         828560.0      7
           AR         822347.0      5
M-SC       XY24       699278.0      6
X-MC       XY24       589233.0      4
X-LC       X40        495777.0      5
           XY24       308657.0      2
X-SC       X40N       298801.0      4
H-SC       OX40N      274616.0      4
           SR         268141.0      2
X-LC       X40N       219532.0      3
X-MC       X40        217516.0      4
H-LC       AR         213519.0      2
H-MC       XY24       183263.0      1
X-MC       XY25       176269.0      2
L-SC       XR         172212.0      2
M-MC       XY24       169953.0      1
X-SC       XY24       164511.0      1
M-SC       AR         164195.0      2
           OX40N      148363.0      4
           XY25       144555.0      1
           XR         133945.0      2
X-MC       X40N       121195.0      1
X-LC       XY25       109556.0      3
M-LC       XR         104704.0      1
H-MC       XR         100185.0      1
M-LC       X5K         96799.0      1
L-SC       OX40N       90223.0      1
X-SC       X40         81334.0      1
H-LC       X200        74022.0      1
H-MC       OX40N       72466.0      1
H-SC       MH          71783.0      1
L-MC       XR          60005.0      1
L-LC       XY25        42681.0      1
M-LC       XY25        40594.0      1
           XY24        28747.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
